# Caso de Estudio: Implementación de una Base de Datos Distribuida en Cassandra para la Plataforma de Pedidos "FastEat"

## Definición del problema

### Descripción del Escenario Empresarial 

"FastEat" es una cadena de restaurantes con presencia en varias ciudades de Latinoamérica. Para incrementar sus ventas y mejorar la experiencia de sus clientes, la empresa ha lanzado una plataforma digital de gestión de pedidos y entregas a domicilio, la cual permite a los usuarios realizar órdenes en línea, personalizar sus platos, rastrear el estado de sus pedidos en tiempo real y calificar el servicio recibido.

La expansión acelerada de la cadena y el éxito de la plataforma han hecho que el volumen de transacciones crezca rápidamente, especialmente en horarios pico y durante promociones especiales. Además, la empresa planea implementar un sistema de recomendaciones personalizadas para sugerir platillos a los clientes según su historial y ubicación, así como análisis en tiempo real de las preferencias de los usuarios.

La plataforma debe operar de manera continua (24/7), soportar miles de operaciones simultáneas y garantizar que la información de pedidos y entregas sea precisa, rápida y esté siempre disponible, incluso en caso de fallos o caídas de algún servidor.

### Problema a Resolver

El sistema de base de datos relacional utilizado inicialmente por FastEat ha comenzado a presentar graves problemas de rendimiento: demoras en la consulta de pedidos durante horas pico, dificultades para escalar la infraestructura ante el crecimiento de la demanda y limitaciones para integrar nuevos módulos como la recomendación personalizada o el análisis de datos en tiempo real.

Por este motivo, la dirección tecnológica de FastEat ha decidido migrar la plataforma a una base de datos distribuida basada en Cassandra, con el objetivo de asegurar escalabilidad, alta disponibilidad y tolerancia a fallos, además de facilitar el manejo eficiente de grandes volúmenes de datos y transacciones concurrentes.

### Objetivos del Proyecto

1. Analizar los requerimientos funcionales del sistema de pedidos y entregas de FastEat, identificando los tipos de datos principales, relaciones y operaciones críticas de consulta y escritura.
2. Diseñar un modelo de datos eficiente para Cassandra, aplicando las mejores prácticas de bases de datos NoSQL, optimizando consultas y garantizando la integridad de la información.
3. Definir la estructura de tablas, claves de partición y clustering keys más adecuadas para responder a los retos de la plataforma (consultas por usuario, seguimiento de pedidos, historial de entregas, etc.).
4. Proponer una estrategia de despliegue en contenedores Docker para la infraestructura de Cassandra, asegurando escalabilidad horizontal y alta disponibilidad. 
5. Justificar las decisiones de diseño en función de los escenarios de uso y los retos operativos descritos.
6. Los datos deben ser leidos (desde python con jupyther) de un archivo externo tipo json para que sean insertados en la tabla  
7. Desarrollar y ejecutar un conjunto de consultas esenciales sobre la base de datos implementada utilizando Python en Jupyter Notebook para la conexión y análisis de los datos. Las consultas deben demostrar la recuperación de pedidos, inserción de nuevos pedidos, actualización del estado de entregas y generación de reportes simples.

### Indicaciones para el Estudiante

Como desarrolladores responsables del proyecto, deberán analizar el escenario presentado y diseñar una solución de base de datos distribuida en Cassandra que cumpla con los requisitos de FastEat. Se espera un documento técnico donde describan el modelo de datos propuesto, justifiquen la elección de las tablas y claves, y detallen la estrategia de despliegue en Docker.

Además, deben preparar y ejecutar, en Jupyter Notebook, consultas relevantes sobre la base de datos usando Python (por ejemplo, empleando la librería `cassandra-driver`). Deberán incluir ejemplos de inserción de pedidos, consultas de estado, actualización de entregas y visualizaciones de datos relevantes.

El enfoque del trabajo debe estar en la eficiencia, escalabilidad y disponibilidad del sistema, demostrando dominio de la integración entre bases de datos distribuidas y herramientas modernas de análisis de datos.

## Docker setup

### Red

```bash
docker network create cassandra-net
```
### Nodos

#### `cassandra-1`

```bash
docker run -d --name cassandra-1 --network cassandra-net \
  --memory=4g --memory-swap=4g \
  -e CASSANDRA_CLUSTER_NAME="UNIR_Cluster" \
  -e CASSANDRA_SEEDS="cassandra-1" \
  -e CASSANDRA_BROADCAST_ADDRESS="cassandra-1" \
  -e CASSANDRA_START_RPC=true \
  -e CASSANDRA_ENDPOINT_SNITCH=GossipingPropertyFileSnitch \
  -e MAX_HEAP_SIZE=1024M \
  -e HEAP_NEWSIZE=256M \
  -p 9042:9042 \
  cassandra

```

#### `cassandra-2`

```bash
docker run -d --name cassandra-2 --network cassandra-net \
  --memory=4g --memory-swap=4g \
  -e CASSANDRA_CLUSTER_NAME="UNIR_Cluster" \
  -e CASSANDRA_SEEDS="cassandra-1" \
  -e CASSANDRA_BROADCAST_ADDRESS="cassandra-2" \
  -e CASSANDRA_ENDPOINT_SNITCH=GossipingPropertyFileSnitch \
  -e MAX_HEAP_SIZE=1024M \
  -e HEAP_NEWSIZE=256M \
  cassandra

```

#### `cassandra-3`

```bash
docker run -d --name cassandra-3 --network cassandra-net \
  --memory=4g --memory-swap=4g \
  -e CASSANDRA_CLUSTER_NAME="UNIR_Cluster" \
  -e CASSANDRA_SEEDS="cassandra-1" \
  -e CASSANDRA_BROADCAST_ADDRESS="cassandra-3" \
  -e CASSANDRA_ENDPOINT_SNITCH=GossipingPropertyFileSnitch \
  -e MAX_HEAP_SIZE=2048M -e HEAP_NEWSIZE=800M \
  cassandra

```

## Jupyter

In [3]:
# Importar las bibliotecas necesarias
from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement
from datetime import datetime

import uuid

### Posible JSON inicial

In [4]:
initial_data = [
    {
        'user': {
            'user_id': 'U00001',
            'name': 'Ana Torres',
            'phone': '0987654321',
            'city': 'Quito',
            'location': {
                'longitude': -78.4678,
                'latitude': -0.1807
            }
        },
        'orders': [
            {
                'order_id': 'O00001',
                'order_timestamp': '2025-07-16T10:12:00',
                'items': [
                    {
                        'item_id': 'I404',
                        'name': 'Pizza Margarita',
                        'quantity': 1,
                        'customizations': [],
                        'price': 10.00,
                        'total': 10.00
                    }
                ],
                'taxes': 1.50,
                'delivery_price': 1.80,
                'total': 13.30,
                'latest': {
                    'status': 'delivered',
                    'location': {
                        'longitude': -78.4678,
                        'latitude': -0.1807
                    },
                    'update_timestamp': '2025-07-16T10:50:00'
                },
                'review': {
                    'comments' : 'Me gustó el servicio!',
                    'rating': 5,
                    'timestamp': '2025-07-16T10:55:00'
                }
            }
        ]
    }
]

### Prueba de conexión

In [5]:
# Conectarse al cluster
cluster = Cluster(['localhost'], port=9042) # Cambiar 'localhost' por la IP del host si no se está ejecutando localmente
session = cluster.connect()

# Ejecutar una consulta simple
row = session.execute("SELECT release_version FROM system.local").one()
if row:
    print(f"Cassandra release version: {row[0]}")
else:
    print("An error occurred.")

Cassandra release version: 5.0.4


### Gestión de Keyspaces

In [6]:
# Listar los keyspaces existentes
rows = session.execute("SELECT keyspace_name FROM system_schema.keyspaces")
for row in rows:
    print(row.keyspace_name)

system_auth
system_schema
fast_eat
system_distributed
system
system_traces


In [9]:
# Crear un keyspace
session.execute("CREATE KEYSPACE IF NOT EXISTS fast_eat WITH replication = {'class': 'SimpleStrategy', 'replication_factor': '3'}")

In [8]:
# Eliminar el keyspace creado
session.execute("DROP KEYSPACE IF EXISTS fast_eat")

In [10]:
# Usar el keyspace 'fast_eat'
session.set_keyspace('fast_eat')

In [11]:
# Crear tipo de dato location
session.execute("""
    CREATE TYPE IF NOT EXISTS location (
        longitude FLOAT,
        latitude FLOAT
    )
""")

### Gestión de Tablas

In [12]:
# Crear tabla de usuarios
session.execute("""
    CREATE TABLE IF NOT EXISTS users (
        user_id TEXT PRIMARY KEY,
        name TEXT,
        phone TEXT,
        city TEXT,
        location FROZEN<location>
    )
""")

In [13]:
class Location:
    def __init__(self, longitude, latitude):
        self.longitude = longitude
        self.latitude = latitude

cluster.register_user_type('fast_eat', 'location', Location)

# Insertar datos iniciales
for data in initial_data:
    user = data['user']
    location = Location(user['location']['longitude'], user['location']['latitude'])
    session.execute("""
        INSERT INTO users (user_id, name, phone, city, location)
        VALUES (%s, %s, %s, %s, %s)
    """, (user['user_id'], user['name'], user['phone'], user['city'], location))

In [14]:
# Mostrar los usuarios insertados
rows = session.execute("SELECT * FROM users")
for row in rows:
    print(f"User ID: {row.user_id}, Name: {row.name}, Phone: {row.phone}, City: {row.city}, Location: ({row.location.longitude}, {row.location.latitude})")

User ID: U00001, Name: Ana Torres, Phone: 0987654321, City: Quito, Location: (-78.4677963256836, -0.18070000410079956)


In [15]:
# Crear tabla de pedidos por usuario en Cassandra
session.execute("""
    CREATE TABLE IF NOT EXISTS orders_by_user (
        user_id TEXT,
        order_id TEXT,
        order_date TIMESTAMP,
        items LIST<TEXT>,
        total FLOAT,
        status TEXT,
        PRIMARY KEY (user_id, order_date, order_id)
    )
""")

## gabi

In [16]:
# Insertar pedidos en la tabla orders_by_user
for data in initial_data:
    user = data['user']
    user_id = user['user_id']
    for order in data['orders']:
        order_id = order['order_id']
        order_date = datetime.fromisoformat(order['order_timestamp'])
        items = [item['name'] for item in order['items']]
        total = order['total']
        status = order['latest']['status']
        session.execute("""
            INSERT INTO orders_by_user (user_id, order_id, order_date, items, total, status)
            VALUES (%s, %s, %s, %s, %s, %s)
        """, (user_id, order_id, order_date, items, total, status))

In [20]:
# Crear tabla principal de pedidos (orders)
session.execute("""
    CREATE TABLE IF NOT EXISTS orders (
        order_id TEXT PRIMARY KEY,
        user_id TEXT,
        order_timestamp TIMESTAMP,
        items LIST<TEXT>,
        taxes FLOAT,
        delivery_price FLOAT,
        total FLOAT,
        status TEXT,
        status_location FROZEN<location>,
        status_update_timestamp TIMESTAMP,
        review_comments TEXT,
        review_rating INT,
        review_timestamp TIMESTAMP,
        city TEXT
    )
""")

# Crear tabla de pedidos por ciudad
session.execute("""
    CREATE TABLE IF NOT EXISTS orders_by_city (
        city TEXT,
        order_date TIMESTAMP,
        order_id TEXT,
        user_id TEXT,
        status TEXT,
        total FLOAT,
        PRIMARY KEY (city, order_date, order_id)
    )
""")

# Crear tabla de reviews por usuario
session.execute("""
    CREATE TABLE IF NOT EXISTS reviews_by_user (
        user_id TEXT,
        review_timestamp TIMESTAMP,
        order_id TEXT,
        review_comments TEXT,
        review_rating INT,
        PRIMARY KEY (user_id, review_timestamp, order_id)
    )
""")

In [21]:
# Insertar datos en la tabla orders
for data in initial_data:
    user = data['user']
    user_id = user['user_id']
    city = user['city']
    for order in data['orders']:
        order_id = order['order_id']
        order_timestamp = datetime.fromisoformat(order['order_timestamp'])
        items = [item['name'] for item in order['items']]
        taxes = order['taxes']
        delivery_price = order['delivery_price']
        total = order['total']
        status = order['latest']['status']
        status_location = Location(order['latest']['location']['longitude'], order['latest']['location']['latitude'])
        status_update_timestamp = datetime.fromisoformat(order['latest']['update_timestamp'])
        review_comments = order['review']['comments']
        review_rating = order['review']['rating']
        review_timestamp = datetime.fromisoformat(order['review']['timestamp'])
        session.execute("""
            INSERT INTO orders (order_id, user_id, order_timestamp, items, taxes, delivery_price, total, status, status_location, status_update_timestamp, review_comments, review_rating, review_timestamp, city)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (order_id, user_id, order_timestamp, items, taxes, delivery_price, total, status, status_location, status_update_timestamp, review_comments, review_rating, review_timestamp, city))

# Insertar datos en la tabla orders_by_city
        session.execute("""
            INSERT INTO orders_by_city (city, order_date, order_id, user_id, status, total)
            VALUES (%s, %s, %s, %s, %s, %s)
        """, (city, order_timestamp, order_id, user_id, status, total))

# Insertar datos en la tabla reviews_by_user
        session.execute("""
            INSERT INTO reviews_by_user (user_id, review_timestamp, order_id, review_comments, review_rating)
            VALUES (%s, %s, %s, %s, %s)
        """, (user_id, review_timestamp, order_id, review_comments, review_rating))

## Consultas esenciales sobre la base de datos

In [22]:
# Recuperar todos los pedidos de un usuario
user_id = 'U00001'
rows = session.execute("SELECT * FROM orders_by_user WHERE user_id=%s", [user_id])
for row in rows:
    print(f"Order ID: {row.order_id}, Date: {row.order_date}, Items: {row.items}, Status: {row.status}, Total: {row.total}")

Order ID: O00001, Date: 2025-07-16 10:12:00, Items: ['Pizza Margarita'], Status: pending, Total: 13.300000190734863


In [23]:
# Consultar pedidos por ciudad y fecha
city = 'Quito'
rows = session.execute("SELECT * FROM orders_by_city WHERE city=%s", [city])
for row in rows:
    print(f"Order ID: {row.order_id}, Date: {row.order_date}, User: {row.user_id}, Status: {row.status}, Total: {row.total}")

Order ID: O00001, Date: 2025-07-16 10:12:00, User: U00001, Status: delivered, Total: 13.300000190734863


In [24]:
# Actualizar estado de entrega de un pedido
order_id = 'O00001'
new_status = 'received'
session.execute("UPDATE orders SET status=%s WHERE order_id=%s", [new_status, order_id])
print(f"Estado actualizado para el pedido {order_id}")

Estado actualizado para el pedido O00001


In [25]:
# Generar reporte simple de reviews por usuario
user_id = 'U00001'
rows = session.execute("SELECT * FROM reviews_by_user WHERE user_id=%s", [user_id])
for row in rows:
    print(f"Order: {row.order_id}, Rating: {row.review_rating}, Comments: {row.review_comments}, Timestamp: {row.review_timestamp}")

Order: O00001, Rating: 5, Comments: Me gustó el servicio!, Timestamp: 2025-07-16 10:55:00
